<a href="https://colab.research.google.com/github/folkn/folkn.github.io/blob/master/Linear_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

references
https://numericalmethodsece101.weebly.com/doolittlersquos-method.html
https://numericmethod.weebly.com/uploads/2/5/0/8/25086597/lkm-8-e.pdf

We start by importing the necessary libraries, namely **numpy** and **scipy** for linear algebra functions. 

***Run Me First!***:

In [0]:
import numpy as np #Import NumPy with the shortcut 'np'
from scipy import linalg #Import Linear Algebra functions from SciPy
np.set_printoptions(floatmode='maxprec_equal') #Set maximum Output precision

# SOLVING LINEAR SYSTEMS in PYTHON

Assignment 1 for LE230 Numerical Methods

**5910751485 Warapong Narongrit**

**5910755627 Techin Chotpongwatana**

We are interested in solving simultaneous linear equations.
FIRST, we input the simultaneous equation in matrix form $Ax=b$ :

$$
\begin{bmatrix}
    A_{11}       & A_{12} & A_{13} & \dots & A_{1n} \\
    A_{21}       & A_{22} & A_{23} & \dots & A_{2n} \\
    A_{d1}       & A_{d2} & A_{d3} & \dots & A_{dn}
\end{bmatrix} 
\begin{bmatrix}
     x_1 \\
     \dots \\
     x_n 
\end{bmatrix}
= \begin{bmatrix}
    b_1 \\
    \dots \\
    b_n
\end{bmatrix}
$$

Input the matrix to be solved here. $A$ as a square matrix, $B$ as a column vector. Make sure none of the diagonals are zero!


In [2]:
#INPUT MATRIX TO BE SOLVED HERE# No Zero diagonals please - enter arrays with decimals please (-100. instead of -100)
A_ = np.array([[1.,-0.25, -0.25, 0],
              [-0.25,1.,0,-0.25],
              [-0.25,0,1.,-0.25],
              [0,-0.25,-0.25,1.]])

b_ = np.array([[50.], 
              [50.], 
              [25.], 
              [25.]])

Print_Values = False #Print value of each step? True or False

print('Input:\nA=\n' + str(A_)); print('b=\n' + str(b_))   #Prints matrices
print('\nOutput:\neigenvalues= ' + str(np.linalg.eigvals(A_))) #Checks eigenvalues
print ('\nActual numpy Solution: X=\n' + str(np.linalg.solve(A_,b_))) #Exact solution as calculated by numpy

Input:
A=
[[ 1.00 -0.25 -0.25  0.00]
 [-0.25  1.00  0.00 -0.25]
 [-0.25  0.00  1.00 -0.25]
 [ 0.00 -0.25 -0.25  1.00]]
b=
[[50.]
 [50.]
 [25.]
 [25.]]

Output:
eigenvalues= [0.5 1.0 1.5 1.0]

Actual numpy Solution: X=
[[87.5]
 [87.5]
 [62.5]
 [62.5]]


## Analytical Solution using Gauss Elimination

The Gauss Elimination Method is a method of using *elementary row operations* until an upper triangle matrix is formed.

The code uses partial pivoting, which is an algorithm of first making the first column all zero (except the first row), and then continuing to the other columns. Since we are not calculating this by hand, we do not have to understand any special techniques, but rather we could just use the standard long approach.


In [3]:
def gauss(A, b):
  
  #Gauss Elim.
  n = A.shape[0] #Number of equations/rows
  for row in range(0, n-1): #For each row
      for i in range(row+1, n): #For each column of the next row
          factor = A[i,row] / A[row,row] #calculate the multiplier (ex. R2 <- R2 - (factor)*R1)
          for j in range(row, n):
              A[i,j] = A[i,j] - factor * A[row,j] #Perform elementary operation on A
          b[i] = b[i] - factor * b[row] #Perform same elementary operation on B
      if Print_Values is True:
        print('\nA =\n' + str(A) + '\nB=\n' + str(b) )
  #Back substitution
  x = np.zeros((n,1)) #Generate solutions vector
  x[n-1] = b[n-1] / A[n-1, n-1] #Solve last row first: a basic division 
  for row in range(n-2, -1, -1): #For all other rows, bottom up
      sums = b[row] #store x values of last row here, and add to this as we consider each row
      for j in range(row+1, n): #For each known column                       (ex. 2x + 3Y=5, if Y is known from previous)
          sums -= A[row,j] * x[j] #Move known values to B side               (ex. 2x + 3(4)=5 --> 2x = 5 - 12 )
      x[row] = sums / A[row,row]  #Divide by unknown coefficient to get sol  (ex.                  x = 7/2    )
  return x
  

x = gauss(np.copy(A_),np.copy(b_))
print ('X=\n' +str(x) +'\tANS')

X=
[[87.5]
 [87.5]
 [62.5]
 [62.5]]	ANS


## Analytical Solution using LU Factorization 

This method uses the idea that all matrices (except those with some diagonal elements being zero) can be factorized into products Lower and upper triangular matrices:

$$A = LU$$ 

$$
\begin{bmatrix}
    A_{11}       & A_{12} & A_{13}  \\
    A_{21}       & A_{22} & A_{23}  \\
    A_{31}       & A_{32} & A_{33} 
\end{bmatrix} =
\begin{bmatrix}
    l_{11}       & 0 & 0  \\
    l_{21}       & l_{22} & 0  \\
    l_{31}       & l_{32} & l_{33} 
\end{bmatrix} 
\begin{bmatrix}
    u_{11}       & u_{12} & u_{13}  \\
    0       & u_{22} & u_{23}  \\
    0       & 0 & 0
\end{bmatrix}
$$

There is more than one way of factorizing, so three **LU Factorization** methods are used:
1. **Doolittle** - all Diagonals of $L$ are $1$    ($l_{11} = l_{22} = l_{nn} = 1$)
2. **Crout** - all Diagonals of $U$ are $1$     ($u_{11} = u_{22} = u_{nn} = 1$)
3. **Cholesky** - $U = L^\text{T}$ or equivalently $L=U^\text{T}$ such that $A=LL^\text{T}$ (*works only if A=positive definite(**positive eigenvalues**) and symmetric **($A=A^\text{T}$)**
---
**The procedure to solving linear systems is:**
1. LU-Factorize the coefficients matrix $A$ using one of the methods
2. Let $LY = b$ where $Y$ is an intermediate variable matrix. Solve for $Y$
3. Let $UX = Y$ then solve for the unknowns $X$ 

(This works because $AX=b, LUX=b, \text{let }Y=UX, \text{so }LY=b$)

I first create functions for steps 2 and 3 since all methods will require these steps.

***Run This First!***

In [0]:
def LY_B (L, B) : #Step 2
    Y =  np.matmul((np.linalg.inv(L)), B) #Y= L^-1 * B
    return Y

def UX_Y (U, Y) : #Step 3
    X = np.matmul(np.linalg.inv(U) , Y) #X=U^-1 * Y
    return X

Now I will perform the three methods of factorization methods

### 1. Doolittle's Method https://www.codewithc.com/lu-decomposition-algorithm-flowchart/ https://vismor.com/documents/network_analysis/matrix_algorithms/S4.SS2.php ###

In the code, we convert the $LU$ vector multiplication into a dot multiplcation problem for better scaling.

In [5]:
#1. Doolittle Factorization https://stackoverflow.com/questions/48715594/why-is-my-profs-version-of-lu-decomposition-faster-than-mine-python-numpy
def doolittle(A) : 
    N = np.size(A, 0)    # Number of Rows (Equations)
    
    # Create empty L U matrices with the dimensions of A
    L = np.zeros_like(A)  
    U = np.zeros_like(A)


    for k in range(N):   #For each equation (row) k
        #Determining Diagonal Elements
        L[k, k] = 1      #Set the L diagonal to 1  (Doolittle Requirement)
        U[k, k] = (A[k, k] - np.dot(L[k, :k], U[:k, k])) 
        #Determining Other Elements
        for j in range(k+1, N):
            U[k, j] = (A[k, j] - np.dot(L[k, :k], U[:k, j])) / L[k, k]
        for i in range(k+1, N):
            L[i, k] = (A[i, k] - np.dot(L[i, :k], U[:k, k])) / U[k, k]
    return L, U
    
L, U = doolittle(A_);
if Print_Values is True:
  print ('L=\n' +str(L)); print('U=\n' + str(U))

#Step 2, 3, ANS
Y= LY_B(L,b_)
X = UX_Y(U,Y)

if Print_Values is True:
  print (('LY=B\nY=\n' + str(Y) + 'UX=Y\n')); 
print ('X=\n' +str(X) +'\tANS')


X=
[[87.5]
 [87.5]
 [62.5]
 [62.5]]	ANS


### 1. Doolittle's Method using SciPy

In [6]:
print('Input:\nA=\n' + str(A_)); print('b=\n' + str(b_))   #Prints matrices
#Doolittle using SciPy's functinos
def scipy_plu(A) :
    L, U = linalg.lu_factor(A)
    return L,U


 #Step 2, 3, ANS
scipy_plu(A_)
Y= LY_B(L,b_)
X = UX_Y(U,Y)
print ('LY=B\nY=\n' + str(Y)); print ('UX=Y\nX=\n' +str(X) +'\tANS')

Input:
A=
[[ 1.00 -0.25 -0.25  0.00]
 [-0.25  1.00  0.00 -0.25]
 [-0.25  0.00  1.00 -0.25]
 [ 0.00 -0.25 -0.25  1.00]]
b=
[[50.]
 [50.]
 [25.]
 [25.]]
LY=B
Y=
[[50.00000000]
 [62.50000000]
 [41.66666667]
 [53.57142857]]
UX=Y
X=
[[87.5]
 [87.5]
 [62.5]
 [62.5]]	ANS


### 2. Crout's Method

In [7]:
#2. Crout Factorization 
def crout(A) :
    N = np.size(A, 0)    # Number of Rows (Equations)
    
    # Create empty L U matrices with the dimensions of A
    L = np.zeros_like(A)  
    U = np.zeros_like(A)


    for k in range(N):   #For each equation (row) k
        #Determining Diagonal Elements
        U[k, k] = 1      #Set the U diagonal to 1  (Crout Requirement)
        L[k, k] = (A[k, k] - np.dot(L[k, :k], U[:k, k])) 
        #Determining Other Elements
        for j in range(k+1, N):
            U[k, j] = (A[k, j] - np.dot(L[k, :k], U[:k, j])) / L[k, k]
        for i in range(k+1, N):
            L[i, k] = (A[i, k] - np.dot(L[i, :k], U[:k, k])) / U[k, k]
    return L, U
    
L, U = crout(A_);
if Print_Values is True:
  print ('L=\n' +str(L)); print('U=\n' + str(U))

#Step 2, 3, ANS
Y= LY_B(L,b_)
X = UX_Y(U,Y)
if Print_Values is True:
  print ('LY=B\nY=\n' + str(Y) + 'UX=Y\n'); 
print ('X=\n' +str(X) +'\tANS')

X=
[[87.5]
 [87.5]
 [62.5]
 [62.5]]	ANS


### 3. Cholesky's Method

In [8]:
#3. Cholesky's Method
def cholesky(A) :
    L = np.linalg.cholesky(A) #wow!
    U = L.T #Cholesky requirement U=transpose(L)
    return L,U

#Try the cholesky, and catch any errors
try:
    L,U = cholesky(A_)
except Exception as err:
    print (err)
if Print_Values is True:
  print ('L=\n' +str(L)); print('U=\n' + str(U))
#Step 2, 3, ANS
Y= LY_B(L,b_)
X = UX_Y(U,Y)
if Print_Values is True:
  print ('LY=B\nY=\n' + str(Y) + 'UX=Y\n');
print ('X=\n' +str(X) +'\tANS')

X=
[[87.5]
 [87.5]
 [62.5]
 [62.5]]	ANS


## Numerical Solutions using Iterations

This method uses repeated calculations to **approximate** the solutions. It is normally used for matrices with large diagonal entries and those with many zero coefficients.

**Input your initial guess, and stopping conditions first:**

In [0]:
###Please Input 'Guess' Here as a column vector###
X_Guess = np.array([[100.],
                    [100.],
                    [100.],
                    [100.]])

Iterations = 100            # Stop calculation if either tolerance or iteration condition is met
Tolerance = 0.000001 #Absolute Tolerance (epsilon)          Set to <=0 to ignore

### 1. Jacobi Iteration
Assumes that the diagnoal elements are non-zero ($a_{nn}\ne 0$)https://www3.nd.edu/~zxu2/acms40390F12/Lec-7.3.pdf https://www.kth.se/social/files/5885d039f2765429974418ce/Lab1.pdf
https://gist.github.com/angellicacardozo/3a0891adfa38e2c4187612e57bf271d1



For a system such as
$$x + y + z = 1 \\2x+4y+8z=2 \\3x+12y+20z=3\\$$
We try to isolate each variable in each equation as such:
$$x=1-y-z(1)\\y=\frac{2-2x-8z}{4}(2)\\z=\frac{3-3x-12y}{20}(3)
$$

Now we come up with an *initial guess* of the solutions, $x^{(0)}$

In our example, let's suppose $[x^{(0)}, y^{(0)}, z^{(0)}]=[0,0,0]$

We substitute the initial guess into $(1),(2),(3)$, which will give us $x^{(1)}$ (our first iteration)

We repeat this step until we are satisfied of our solutions.

**In matrix form**, this is shown as

$$A=\begin{bmatrix}
    1       & 1 & 1  \\
    2       & 4 & 8  \\
    3       & 12 & 20 
\end{bmatrix}
, B=
\begin{bmatrix}
    1      \\
    2      \\
    3     
\end{bmatrix}
$$


Then we attempt to create equations (1) (2) (3)  by separating the diagonals

$$
D=\begin{bmatrix}
    1      \\
    4      \\
    12     
\end{bmatrix} \text{(Diagonal elements)}
$$ 

$$
R=\begin{bmatrix}
    1       & 1 & 1  \\
    2       & 4 & 8  \\
    3       & 12 & 20 
\end{bmatrix} -
\begin{bmatrix}
    1       & 0& 0  \\
    0       & 4 & 0 \\
    0       & 0 & 20 
\end{bmatrix}
=\begin{bmatrix}
    0       & 1 & 1  \\
    2       & 0 & 8  \\
    3       & 12 & 0 
\end{bmatrix}
$$

$$
x^{(n)} = \frac{(RX - B)}{D} \text{  (elementwise division)}
$$
In code, the matrix multiplication RX is performed row-wise as a dot product problem to simplify code.

The initial guess and iterations are placed in matrix $x$ and substituted as above.


If the diagonals of $A$ are all $1$, the iteration can be written as $$x^{(n+1)}=B+(I-A)x^{(m)}$$. Otherwise, it is written as 
$$x_i^{(n)}=\frac{1}{a_{ii}}[\sum^{eq}_{j=1, j\ne i}(-a_{ij}x^{(n)})+B]$$



In [10]:
def jacobi(A, b, N, x, tol):  
    print('Initial Guess X^(0)=\n'+str(x.flatten()) + '\nfor A=\n' +str(A_) + '  \nJacobi using ' +str(Iterations) + ' Iterations, Abs.Tol epsilon =' + str(Tolerance))
    eq = np.size(A, 0)    # Number of Rows (Equations)
    D = np.diag(A) #Obtain the diagonals as a row vector
    R = A - np.diagflat(D) #Obtain matrix A without its diagonals
                                                                                                                                                                    
    for i in range(N): #Iterations
        x_old = x #store previous iteration
        x_max=0 #For error condition
        
        x = (b - np.matmul(R,x)) / D #new iteration
        
        #Error Calculation
        if np.amax(abs(x-x_old)) > x_max: #Determines the maximum error from all variables compared to prev. iter
          x_max = np.amax(abs(x-x_old)) #Find max error among unknowns X 
          
        if Print_Values is True: #Print values of each Iteration
          if i is 0:
            print('\n\nIteration, [Iteration Values], \t\t\t\t Max Abs Error')  
          print ('('+str(i+1)+ ')\t' + str(x) + '\t\t' + str(x_max))
       
      #Checks stopping condition
        if x_max < tol: #If maximum error is less than epsilon (stopping condition)
          print('Tolerance condition met before number of iterations', i+1, x_max)
          return x, i+1., x_max #Then stop calculation
        
        
    print('Number of iterations condition met before tolerance condition', i, x_max)
    return x, i, x_max
   


sol,iterN,x_max = jacobi(A_,b_.flatten(), Iterations, X_Guess.flatten() , Tolerance)
print('\n\nX='+ str(sol) + '\nwith a final epsilon of ' + str(x_max) + ' after ' + str(iterN) + ' iterations')

Initial Guess X^(0)=
[100. 100. 100. 100.]
for A=
[[ 1.00 -0.25 -0.25  0.00]
 [-0.25  1.00  0.00 -0.25]
 [-0.25  0.00  1.00 -0.25]
 [ 0.00 -0.25 -0.25  1.00]]  
Jacobi using 100 Iterations, Abs.Tol epsilon =1e-06
('Tolerance condition met before number of iterations', 25, 7.450580596923828e-07)


X=[87.50000075 87.50000075 62.50000075 62.50000075]
with a final epsilon of 7.450580596923828e-07 after 25.0 iterations


### 2. Gauss-Seidel Iteration
https://austingwalters.com/gauss-seidel-method/
http://milesbarnhart.com/portfolio/python/python-gauss-seidel-approximation-method/

With the Jacobi method, we use the same iteration values $x^{(n)}$ until we are done with the iteration.

With the Gauss-Seidel method, we use the newly calculated value $x^{(n+1)}_i$ right after it is known. For example, once we determined $x_1^{(n+1)}$ from the first equation, we use it to find $x_2^{(n+1)}$ in the second equation right away. 

This is similar to the previous method, but requires certain modifications to the equation.

In matrix form, we wil have to separate the lower and upper triangles of matrix $A$ to account for this change. The matrix equation becomes[1] $$x^{(n+1)} = B - L x^{(n+1)} - Ux^{(n)}$$ 

Our code will rearrange this equation and output the values at the end of each iteration

$$x^{(n+1)} + Lx^{(n+1)} = B - Ux^{(n)}$$

$$x^{(n+1)}(I+L) = B-Ux^{(n)}$$

($I + \text{strictly lower matrix} = \text{lower matrix with diagonals}$ )

$$x^{(n+1)} = L^{-1}(B-Ux^{(n)})$$

In [11]:
def gauss_seidel(A, b, N, x, tol):
    print('Initial Guess X^(0)= '+str(X.flatten()) + '\nfor A=\n' +str(A_) + '  \nGauss-Seidel using ' +str(Iterations) + ' Iterations, Abs.Tol epsilon =' + str(Tolerance))

    L = np.tril(A) #Lower triangle
    U = np.triu(A, 1) #Upper triangle, 1 extracts strictly upper triangle
    
    for i in range(N):
        x_old = x #store previous iteration
        x_max=0 #For error condition
        
        x = np.matmul(np.linalg.inv(L), b - np.matmul(U, x)) #new Iteration
        
        #Error Calculation
        if np.amax(abs(x-x_old)) > x_max: #Determines the maximum error from all variables compared to prev. iter
          x_max = np.amax(abs(x-x_old)) #Find max error among unknowns X 
          
        if Print_Values is True: #Print values of each Iteration
          if i is 0:
            print('\n\nIteration, [Iteration Values], \t\t\t\t Max Abs Error')  
          print ('('+str(i+1)+ ')\t' + str(x) + '\t\t' + str(x_max))
       
      #Checks stopping condition
        if x_max < tol: #If maximum error is less than epsilon (stopping condition)
          print('Tolerance condition met before number of iterations', i+1, x_max)
          return x, i+1., x_max #Then stop calculation
        
        
    print('Number of iterations condition met before tolerance condition', i, x_max)
    return x, i, x_max



sol,iterN,x_max = gauss_seidel(A_, b_.flatten(), Iterations, X_Guess.flatten(), Tolerance )
print('\n\nX='+ str(sol) + '\nwith a final epsilon of ' + str(x_max) + ' after ' + str(iterN) + ' iterations')

Initial Guess X^(0)= [87.5 87.5 62.5 62.5]
for A=
[[ 1.00 -0.25 -0.25  0.00]
 [-0.25  1.00  0.00 -0.25]
 [-0.25  0.00  1.00 -0.25]
 [ 0.00 -0.25 -0.25  1.00]]  
Gauss-Seidel using 100 Iterations, Abs.Tol epsilon =1e-06
('Tolerance condition met before number of iterations', 15, 2.7939677238464355e-07)


X=[87.50000009 87.50000005 62.50000005 62.50000002]
with a final epsilon of 2.7939677238464355e-07 after 15.0 iterations


# Speed Comparisons
The following function measures the time taken for all the calculations: Press run-all first before running this

In [12]:
from scipy.stats import rankdata #Import ranking method from scipy
import time, sys, os #Import timer, python interpreter controls, OS controls (command line)

class suppressPrint: #Supress printoutput
    def __enter__(self): #Upon entering the class, suppress console output
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb): #Upon exiting class, enable console output
        sys.stdout.close()
        sys.stdout = self._original_stdout 
        
with suppressPrint(): #Using suppress print class
  tictoc = np.zeros(7) #Create np array to store time
  iters = ['','','','','',0,0] #Number of iterations
  names = [' Gauss Elimination','      LU-Doolittle','LU-Doolittle-SciPy', '          LU-Crout',
           '       LU-Cholesky', '       Iter-Jacobi',' Iter-Gauss-Seidel']
  
  t = time.time()
  gauss(np.copy(A_),np.copy(b_)) #Gauss Elim
  tictoc[0] = time.time() - t

  t= time.time()
  doolittle(A_); #Doolittle
  LY_B(L,b_)
  UX_Y(U,Y)
  tictoc[1] = time.time() - t

  t= time.time()
  scipy_plu(A_) #Doolittle SciPy
  LY_B(L,b_)
  UX_Y(U,Y)
  tictoc[2] = time.time() - t

  t= time.time()
  crout(A_); #Crout
  LY_B(L,b_)
  UX_Y(U,Y)
  tictoc[3] = time.time() - t

  t= time.time()
  try:
      cholesky(A_) #Cholesky
  except Exception as err:
      pass
  LY_B(L,b_)
  UX_Y(U,Y)
  tictoc[4] = time.time() - t


  t= time.time()
  sol,iters[5],x_max = jacobi(A_,b_.flatten(), Iterations, X_Guess.flatten() , Tolerance) #Jacobi
  tictoc[5] = time.time() - t

  t= time.time()
  sol,iters[6],x_max = gauss_seidel(A_, b_.flatten(), Iterations, X_Guess.flatten(), Tolerance ) #Gauss Seiel
  tictoc[6] = time.time() - t
  
tictocrank = rankdata(tictoc) #Rank time from fastest to slowest
print("'      Method     ' , Rank,  Time (ms)     ,   No. Iterations")
for i in range(len(tictoc)):
  print(names[i], int(tictocrank[i]), tictoc[i]*1000, iters[i])

'      Method     ' , Rank,  Time (ms)     ,   No. Iterations
(' Gauss Elimination', 4, 0.19788742065429688, '')
('      LU-Doolittle', 5, 0.3151893615722656, '')
('LU-Doolittle-SciPy', 2, 0.11992454528808594, '')
('          LU-Crout', 3, 0.15020370483398438, '')
('       LU-Cholesky', 1, 0.04887580871582031, '')
('       Iter-Jacobi', 7, 1.5561580657958984, 25.0)
(' Iter-Gauss-Seidel', 6, 1.3570785522460938, 15.0)


# References
[1] E. Kreyszic. "Numerical Linear Algebra". *Advanced Engineering Analysis* pp.845-863. 9e
[2] https://learnche.org/3E4/Assignment_2_-_2010_-_Solution/Bonus_question 
